#Installation

In [1]:
pip install scikit-learn

In [2]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=0f700b1c576e878452685244eed395494c3edabeb4e25bd8b541ac9f92c1a703
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
pip install plotly

In [9]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score
from sentence_transformers import SentenceTransformer
import numpy as np

'''
Variables:
---------

corpus : list of documents
embeddings : documents embeddings of size NxM (N : number of documents, M : embedding dimension)
red_emd : reduced embeddings matrix using dimentionality reduction
k : number of clusters
labels : documents labels
pred : list of clustering predicted clusters

''';

#reduction de la dimension

In [5]:
def dim_red(mat, p):
  '''
  Perform dimensionality reduction

  Input:
  -----
  mat : NxM list
  p : number of dimensions to keep
  Output:
  ------
  red_mat : NxP list such that p<<m
  '''
  # Standardisez les données pour obtenir des résultats plus significatifs
  scaler = StandardScaler()
  data_std = scaler.fit_transform(mat)

  # Instanciez le modèle t-SNE
  tsne_model = TSNE(n_components=p, random_state=42)

  # Ajustez le modèle aux données
  resultat_tsne = tsne_model.fit_transform(data_std)

  # Les résultats sont maintenant dans tsne_result
  # Vous pouvez créer un DataFrame pour visualiser les résultats
  tsne_df = pd.DataFrame(resultat_tsne, columns=[f"Dimension_{i}" for i in range(1, p + 1)])
  red_mat = tsne_df

  return red_mat

#clustering

In [6]:
def clust(mat, k):
    '''
    Perform clustering

    Input:
    -----
        mat : input list
        k : number of cluster
    Output:
    ------
        pred : list of predicted labels
    '''
    model_kmeans = KMeans(n_clusters=k, random_state=42)

    # Ajustez le modèle aux données
    model_kmeans.fit(mat)


    pred = model_kmeans.labels_

    return pred

#run et test NMI ARI

In [7]:
# import data
ng20 = fetch_20newsgroups(subset='test')
corpus = ng20.data[:2000]
labels = ng20.target[:2000]
k = len(set(labels))

# embedding
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(corpus)

# perform dimentionality reduction
red_emb = dim_red(embeddings, 3)

# perform clustering
pred = clust(red_emb, k)

# evaluate clustering results
nmi_score = normalized_mutual_info_score(pred,labels)
ari_score = adjusted_rand_score(pred,labels)

print(f'NMI: {nmi_score:.2f} \nARI: {ari_score:.2f}')


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


NMI: 0.40 
ARI: 0.21


#Visualisation des clusters

In [10]:
# Visualiser les résultats en 3D avec Plotly
fig = px.scatter_3d(red_emb, x='Dimension_1', y='Dimension_2', z='Dimension_3', color=pred, title='t-SNE Visualisation en 3D')
fig.show()